# Trabalho Final

**Aluno**: Lucas de Miranda Rodrigues

**Matrícula**: 96143


## 03 - Tratamento da base *formacao.csv*

Essa base conta com 220 registros de formação acadêmica dos 111 perfis, portanto um perfil pode ter mais de uma formação. E essa tem uma coluna de datas (**dates**), registrando o período de ínicio e de fim da formação dos perfis. Outra coluna presente é o nome do título ou diploma (**degree**) adquirido pelo perfil em sua formação. Por fim, encontram-se as colunas da área estudada (**major**) e da instituição cursada(**school**) pelos perfis.

Deve-se atentar a presença de dados ausentes/faltantes nessas colunas menciondas acima, sendo necessiário uma análise mais cuidadosa dessa base e fazendo as devidas filtragens dos mesmos.    

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

formacao = pd.read_csv("bases/formacao.csv")

# Análise inicial da qualidade dessa base
print("Quantidade de registros: ", len(formacao))
print("Quantidade de valores distintos de perfil_id: ", formacao.perfil_id.nunique())
print("Dados faltantes em degree: ", len(formacao.loc[formacao.degree.isnull()]))
print("Dados faltantes em date: ", len(formacao.loc[formacao.date.isnull()]))
print("Dados faltantes em major: ", len(formacao.loc[formacao.major.isnull()]))
print("Dados faltantes em school: ", len(formacao.loc[formacao.school.isnull()]))
print("Dados faltantes em degree + major: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull())]))
print("Dados faltantes em degree + major + school: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull()) & (formacao.school.isnull())]))
print("Valores distintos em degree: ", len(formacao.groupby('degree').nunique()))
print("Valores distintos em major: ", len(formacao.groupby('major').nunique()))
print("Valores distintos em school: ", len(formacao.groupby('school').nunique()))

Quantidade de registros:  220
Quantidade de valores distintos de perfil_id:  111
Dados faltantes em degree:  29
Dados faltantes em date:  7
Dados faltantes em major:  50
Dados faltantes em school:  1
Dados faltantes em degree + major:  13
Dados faltantes em degree + major + school:  1
Valores distintos em degree:  135
Valores distintos em major:  114
Valores distintos em school:  122


In [2]:
formacao.head(10)

,date,degree,major,school,perfil_id
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5


## 03.1 - Tratamento das data

Um primeiro tratamento feito será o de data. A coluna **date** que indica o íncio e o fim do curso e será quebrada em duas colunas numéricas (**inicio** e **fim**), e ainda numa terceira coluna indicando a duração estimada do curso em anos baseado na informação obtida (**duracao**).

Como o formato da data está: **'20XX - 20XX'**, usou-se a função `split` para separar essa string, gerando uma resposta que está armazenada na coluna **ano_controle**. Depois, fez a devida separação dos anos em ínicio e fim.

In [3]:
#Separa a string
formacao['ano_controle']=formacao.date

#Armazena a resposta na coluna: ano_controle
formacao.ano_controle=formacao.date.str.split(' ')

def separacao_ano(df,posicao):
    """Função que separa o ano de início e
    o de fim."""
    vetor=np.arange(len(df))
    TF_null=df.notnull()
    vetora=np.nan*vetor
    for x in vetor:
        if TF_null.loc[x]==True:
            if len(df.loc[x])==3:
                vetora[x]=int(df.loc[x][posicao])
            else:
                vetora[x]=int(df.loc[x][0])
    return vetora

#Armazena valor de inicio e de fim
formacao['inicio']=separacao_ano(formacao.ano_controle,0)
formacao['fim']=separacao_ano(formacao.ano_controle,2)

#Calcula a duração da formação em anos
formacao['duracao']=formacao.fim-formacao.inicio

#Filtra os dados ausentes nas colunas de inicio, fim e duração
formacao.inicio.fillna("Não Informado", inplace=True)
formacao.fim.fillna("Não Informado", inplace=True)
formacao.duracao.fillna("Não Informado", inplace=True)

#Mostra o resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0


## 03.2 - Tratamento da coluna de grau de escolaridade

Uma vez que a coluna **degree** não é muito precisa, uma coluna **grau** será criada. Ela começará vazia e será armazenado a inferência do grau de formação baseado na análise das demais colunas.

Uma primeira estratégia será avaliar o nome do **degree** para inferir se ele contém algum fragmento que possa indicar algum dos seguintes graus de escolaridade, representados por números: **Ensino Médio**=1, **Técnico**=2, **Superior**=3 ou **Pós-Graduação**=4.

Para isso, utilize a função [`contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html).

A coluna **grau_controle** contem os valores gerados pela funcão `replace`.

In [4]:
#Gerando a coluna: grau_controle
formacao['grau_controle']=formacao.degree
formacao.grau_controle.fillna("Não Informado", inplace=True)
formacao.grau_controle=formacao.grau_controle.str.replace("Nome do diploma\n", "")

#Vetores com as palavras-chaves para a função contains
vetor_1=['Médio','médio','High']
vetor_2=['Technician','Técnico','Technical','tecnico','Technologist','Tecnólogo','Tecnico','Analista','Help',\
        'Google']
vetor_3=['Bacharel','Bachelor','Graduação','Bacharelado','Study','Cursando','Non',\
         'Interniship','Graduate','Interchange','Bacharelando','B.','Engenharia','Psicóloga',\
         'História','Turismo','Ciências','Ciência','Administração','Economics','Propaganda',\
         'Psicologia','Administracao','bacharelado','Sistemas','Engenheiro','Processos','Comercio','College']
vetor_4=['Executive','Master','Masters','MBA','Pós','Specialist','Loyola','Mestrado','Post','Minor',\
        'Especialista', 'Especialização','Specialization','Marketing','Gestão']


def troca_num_pt1(vetor_string, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    r_vetor=np.zeros(len(df_controle))
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

def troca_num_pt2(vetor_string, r_vetor, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

#Gera a coluna:grau
p1=troca_num_pt1(vetor_1,formacao.grau_controle,1)
p2=troca_num_pt2(vetor_2,p1,formacao.grau_controle,2)
p3=troca_num_pt2(vetor_3,p2,formacao.grau_controle,3)
formacao['grau']=troca_num_pt2(vetor_4,p3,formacao.grau_controle,4)

#Mostra o resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0


Para os demais casos do grau de escolaridade não cobertos pela análise do texto, deverá ser feita uma análise mais profunda. Tente encontrar outros padrões para definir o grau de escolaridade, por exemplo, o nome da instituição de ensino (campo **school**) ou, então, o tempo de duração.

In [5]:
#Utilizando a coluna school e duração para os resultados que não foram cobertos:
#Para o grau 1:
formacao.loc[(formacao.school.str.contains('Colegio')) & (formacao.grau == 0),'grau']=1
formacao.loc[(formacao.school.str.contains('Colégio')) & (formacao.grau == 0),'grau']=1
formacao.loc[(formacao.school.str.contains('Marista')) & (formacao.grau == 0),'grau']=1
formacao.loc[(formacao.school.str.contains('Loyola')) & (formacao.grau == 0),'grau']=1
formacao.loc[(formacao.school.str.contains('Lourenço')) & (formacao.grau == 0),'grau']=1
#Para a grau 2
formacao.loc[(formacao.school.str.contains('ETFG')) & (formacao.grau == 0),'grau']=2
#Para a grau 3
formacao.loc[(formacao.school.str.contains('Universidade')) & (formacao.grau == 0),'grau']=3
formacao.loc[(formacao.school.str.contains('Universitário')) & (formacao.grau == 0),'grau']=3
formacao.loc[(formacao.school.str.contains('Federal')) & (formacao.grau == 0),'grau']=3
formacao.loc[(formacao.school.str.contains('Stuttgart')) & (formacao.grau == 0),'grau']=3
#Para a grau 4
formacao.loc[(formacao.school.str.contains('Stanford')) & (formacao.grau == 0),'grau']=4
formacao.loc[(formacao.school.str.contains('Ecommerce')) & (formacao.grau == 0),'grau']=4

#Mostra o resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0


## 03.3 - Tratamento da coluna de grau de escolaridade

A coluna **school** apresenta uma vasta de gama de categorias. Porém numa análise visual dos seus diferentes valores é possível ver que algumas instituições de ensino apresentam variações de nome. Foi feita uma substituição para agrupar esses casos num único nome. O resultao do tratamento é armazenado na nova coluna **instituicao**.

In [6]:
#Gera a coluna: instituição
formacao['instituicao']=formacao.school

#Filtra para eliminar dados ausentes
formacao.instituicao.fillna('Não Informado',inplace=True)

#Evita variações no nome das instituições
formacao.loc[formacao.instituicao.str.contains('Uni-', case=False),'instituicao']='Centro Universitário de Belo Horizonte'
formacao.loc[formacao.instituicao.str.contains('UNI-', case=False),'instituicao']='Centro Universitário de Belo Horizonte'
formacao.loc[formacao.instituicao.str.contains('UNA', case=False),'instituicao']='Centro Universitário UNA'
formacao.loc[formacao.instituicao.str.contains('UFMG', case=False),'instituicao']='Universidade Federal de Minas Gerais'
formacao.loc[formacao.instituicao.str.contains('Antônio', case=False),'instituicao']='Colégio Santo Antônio'
formacao.loc[formacao.instituicao.str.contains('COTEMIG', case=False),'instituicao']='COTEMIG'
formacao.loc[formacao.instituicao.str.contains('Cotemig', case=False),'instituicao']='COTEMIG'
formacao.loc[formacao.instituicao.str.contains('Sá', case=False),'instituicao']='Faculdade Estácio de Sá'
formacao.loc[formacao.instituicao.str.contains('Getulio', case=False),'instituicao']='Fundação Getulio Vargas'
formacao.loc[formacao.instituicao.str.contains('IFAM', case=False),'instituicao']='Instituto Federal de Educação, Ciência e Tecnologia do Amazonas'
formacao.loc[formacao.instituicao.str.contains('PUC Minas', case=False),'instituicao']='Pontifícia Universidade Católica de Minas Gerais'
formacao.loc[formacao.instituicao.str.contains('SENAC', case=False),'instituicao']='SENAC-MG'
formacao.loc[formacao.instituicao.str.contains('Senac', case=False),'instituicao']='SENAC-MG'
formacao.loc[formacao.instituicao.str.contains('Facamp', case=False),'instituicao']='Faculdade de Campinas'
formacao.loc[formacao.instituicao.str.contains('Fucapi', case=False),'instituicao']='Faculdade FUCAPI'
formacao.loc[formacao.instituicao.str.contains('Fereral', case=False),'instituicao']='Universidade Federal do Amazonas'
formacao.loc[formacao.instituicao.str.contains('Ufam', case=False),'instituicao']='Universidade Federal do Amazonas'
formacao.loc[formacao.instituicao.str.contains('of Amazonas', case=False),'instituicao']='Universidade Federal do Amazonas'
formacao.loc[formacao.instituicao.str.contains('UFAM', case=False),'instituicao']='Universidade Federal do Amazonas'
formacao.loc[formacao.instituicao.str.contains('Nokia', case=False),'instituicao']='Fundação Nokia de Ensino'
formacao.loc[formacao.instituicao.str.contains('PUC-SP', case=False),'instituicao']='Pontifícia Universidade Católica de São Paulo'
formacao.loc[formacao.instituicao.str.contains('Paulista', case=False),'instituicao']='Universidade Paulista'

#Mostra o resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau,instituicao
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0,Stanford University
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0,Universidade Federal de Minas Gerais
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0,Centro Universitário Newton Paiva
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0,Universidade do Estado de Minas Gerais
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0,Universidade Federal de Minas Gerais
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0,Universidade Federal de Minas Gerais
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0,Centro Federal de Educação Tecnológica de Mina...
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0,Universidade Federal de Minas Gerais
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0,California State University-Long Beach
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0,Universidade FUMEC


## 03.4 - Área de Formação

Agrupe as formações em grupos mais macros, por exemplo: Engenharia de Alimento, Engharia Elétrica estariam no grupo de engenharias, já Ciência da Computação, Sistemas de Informação poderiam estar no grupo de T.I. Crie áreas que façam sentido, agrupe os dados criando uma nova coluna no dataframe chamada area.

In [7]:
# Gera a coluna:ano_controle
formacao['area_controle']=formacao.major
formacao.area_controle.fillna("Não Informado", inplace=True)
formacao.area_controle=formacao.area_controle.str.replace("Área de estudo\n", "")

#Vetores com as palavras-chaves para a função contains com os dados da area
v_GES=['Gestão','Administração','Entrepreneurship','Business','Project','Consultoria','Human','Management']
v_FIN=['Economics','Economia','Finance','Finanças','econômicas','Financeira','Contabilidade','Atuariais']
v_TI=['Technoloy','Sysmtems','Informatics','Informatic','Information','Computação','Informatica', 'Informática','Computer',\
     'Systems','Informação','Data','Softwaretechnik','sistemas','Tecnologia','TI','Interaction','Interação']
v_ENG=['Engineering','Engenharia','Aeronáutica','Exatas','Engineer','Ingénierie']
v_COM=['Propaganda','Comunicação','Jornalismo','Public Relations','Mídias', 'Mídia','Marketing','Relações Públicas',\
      'Publicidade']
v_DIR=['Direito']
v_DESAR=['Design','Arts']
v_CIN=['Cinematográfica']
v_HI=['História']
v_TU=['Tourism']
v_PSI=['Psicologia','Psicanálise']
v_EST=['High','Ensino']


def troca_string_pt1(vetor_string, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    r_vetor= [np.nan for x in range(len(df_controle))]
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

def troca_string_pt2(vetor_string, r_vetor, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

#Forma coluna: area com os dados da area
p1=troca_string_pt1(v_DESAR,formacao.area_controle,'Design / Arte')
p2=troca_string_pt2(v_TI,p1,formacao.area_controle,'TI')
p3=troca_string_pt2(v_ENG,p2,formacao.area_controle,'Engenharia')
p4=troca_string_pt2(v_GES,p3,formacao.area_controle,'Gestão / Negócios')
p5=troca_string_pt2(v_DIR,p4,formacao.area_controle,'Direito')
p6=troca_string_pt2(v_COM,p5,formacao.area_controle,'Comunicação / Publicidade')
p7=troca_string_pt2(v_HI,p6,formacao.area_controle,'História')
p8=troca_string_pt2(v_PSI,p7,formacao.area_controle,'Psicologia')
p9=troca_string_pt2(v_EST,p8,formacao.area_controle,'Ensino Médio / Técnico')
p10=troca_string_pt2(v_CIN,p9,formacao.area_controle,'Cinema')
p11=troca_string_pt2(v_TU,p10,formacao.area_controle,'Turismo')
p12=troca_string_pt2(v_FIN,p11,formacao.area_controle,'Economia / Finanças')

#Vetores com as palavras-chaves para a função contains com os dados do grau
v_ENS_grau=['Loyola','Ensino']
v_GES_grau=['Entrepreneurship','Administração','Administracao','Administration','vendas','Comercio']
v_FIN_grau=['Econômicas','Economics']
v_TI_grau=['Sistemas','Eletrônica','Help','Computation','Dados','dados','Informática','Interchange','Integrado']
v_ENG_grau=['Engenharia','Engineering','Engenheiro']
v_COM_grau=['Comunicação','Publicidade','MARKETING','Marketing']
v_PSI_grau=['Psicologia']
v_GAS_grau=['Gastronomia']
v_CIN_grau=['Filmmaking']
v_TU_grau=['Turismo']
v_ID_grau=['English']

#Forma coluna: area com os dados do grau
p13=troca_string_pt2(v_ENS_grau,p12,formacao.grau_controle,'Ensino Médio /Técnico')
p14=troca_string_pt2(v_GES_grau,p13,formacao.grau_controle,'Gestão / Negócios')
p15=troca_string_pt2(v_TI_grau,p14,formacao.grau_controle,'TI')
p16=troca_string_pt2(v_ENG_grau,p15,formacao.grau_controle,'Engenharia')
p17=troca_string_pt2(v_COM_grau,p16,formacao.grau_controle,'Comunicação / Publicidade')
p18=troca_string_pt2(v_GAS_grau,p17,formacao.grau_controle,'Gastronomia')
p19=troca_string_pt2(v_CIN_grau,p18,formacao.grau_controle,'Cinema')
p20=troca_string_pt2(v_TU_grau,p19,formacao.grau_controle,'Turismo')
p21=troca_string_pt2(v_ID_grau,p20,formacao.grau_controle,'Idioma')
p22=troca_string_pt2(v_FIN_grau,p21,formacao.grau_controle,'Economia / Finanças')
formacao['area']=troca_string_pt2(v_PSI_grau,p22,formacao.grau_controle,'Psicologia')

#Filtra dado ausentes
formacao.area.fillna("Não Informado", inplace=True)

#Mostra resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau,instituicao,area_controle,area
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0,Stanford University,Não Informado,Gestão / Negócios
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0,Universidade Federal de Minas Gerais,Economics,Economia / Finanças
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0,Centro Universitário Newton Paiva,Gestão de Negócios,Gestão / Negócios
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0,Universidade do Estado de Minas Gerais,Design de Interiores,Design / Arte
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0,Universidade Federal de Minas Gerais,Computer Technology/Computer Systems Technology,TI
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0,Universidade Federal de Minas Gerais,"Informatics, COLTEC",TI
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0,Centro Federal de Educação Tecnológica de Mina...,Não Informado,Engenharia
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0,Universidade Federal de Minas Gerais,Chemical Engineering,Engenharia
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0,California State University-Long Beach,Chemical Engineering,Engenharia
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0,Universidade FUMEC,Economia e Gestão Empresarial,Economia / Finanças


## 03.5 - Escolaridade

Uma informação consolidada de cada funcionário que pode ser derivada dessa base é a sua escolaridade (**escolaridade**), que é pode ser obtida pegando o maio grau daquele perfil. Gere uma base à parte contendo essa informação.

In [8]:
#Forma base com os dados da escolaridade máxima 
max_esc=formacao.groupby('perfil_id').max()
max_esc['key']=max_esc.index
max_esc['max_grau']=max_esc.grau
max_esc=max_esc.drop(['grau_controle','instituicao','area_controle','area','grau'],axis=1)

#Mostra resultado
max_esc.head(10)

,key,max_grau
perfil_id,,
0,0,4.0
1,1,4.0
2,2,3.0
3,3,3.0
4,4,3.0
5,5,4.0
6,6,4.0
7,7,3.0
8,8,4.0


In [9]:
#Faz a junção dos valores máximos de escolaridade com a base: formacao
formacao['key']=formacao.perfil_id
formacao=pd.merge(formacao,max_esc,how='left',on=['key'])

#Mostra resultados
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau,instituicao,area_controle,area,key,max_grau
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0,Stanford University,Não Informado,Gestão / Negócios,0,4.0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0,Universidade Federal de Minas Gerais,Economics,Economia / Finanças,0,4.0
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0,Centro Universitário Newton Paiva,Gestão de Negócios,Gestão / Negócios,1,4.0
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0,Universidade do Estado de Minas Gerais,Design de Interiores,Design / Arte,1,4.0
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0,Universidade Federal de Minas Gerais,Computer Technology/Computer Systems Technology,TI,2,3.0
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0,Universidade Federal de Minas Gerais,"Informatics, COLTEC",TI,2,3.0
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0,Centro Federal de Educação Tecnológica de Mina...,Não Informado,Engenharia,3,3.0
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0,Universidade Federal de Minas Gerais,Chemical Engineering,Engenharia,4,3.0
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0,California State University-Long Beach,Chemical Engineering,Engenharia,4,3.0
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0,Universidade FUMEC,Economia e Gestão Empresarial,Economia / Finanças,5,4.0


In [10]:
#Descubrindo a área e a duração dos valores máximos de escolaridade
formacao['max_area']=np.nan
formacao['max_duracao']=np.nan
formacao.loc[formacao.grau == formacao.max_grau,'max_area']=formacao.area
formacao.loc[formacao.grau == formacao.max_grau,'max_duracao']=formacao.duracao

#Mostra resultado
formacao.head(10)

,date,degree,major,school,perfil_id,ano_controle,inicio,fim,duracao,grau_controle,grau,instituicao,area_controle,area,key,max_grau,max_area,max_duracao
0,2016 – 2016,"Nome do diploma\nExecutive Education, Innovati...",NaN,Stanford University,0,"[2016, –, 2016]",2016,2016,0,"Executive Education, Innovation & Entrepreneur...",4.0,Stanford University,Não Informado,Gestão / Negócios,0,4.0,Gestão / Negócios,0
1,2007 – 2010,Nome do diploma\nEconomics,Área de estudo\nEconomics,Universidade Federal de Minas Gerais,0,"[2007, –, 2010]",2007,2010,3,Economics,3.0,Universidade Federal de Minas Gerais,Economics,Economia / Finanças,0,4.0,NaN,NaN
2,2015 – 2015,Nome do diploma\nMaster of Business Administra...,Área de estudo\nGestão de Negócios,Centro Universitário Newton Paiva,1,"[2015, –, 2015]",2015,2015,0,Master of Business Administration (MBA),4.0,Centro Universitário Newton Paiva,Gestão de Negócios,Gestão / Negócios,1,4.0,Gestão / Negócios,0
3,2009 – 2012,Nome do diploma\nBacharel em Design de Ambientes,Área de estudo\nDesign de Interiores,Universidade do Estado de Minas Gerais,1,"[2009, –, 2012]",2009,2012,3,Bacharel em Design de Ambientes,3.0,Universidade do Estado de Minas Gerais,Design de Interiores,Design / Arte,1,4.0,NaN,NaN
4,2016 – 2019,Nome do diploma\nBachelor’s Degree,Área de estudo\nComputer Technology/Computer S...,Universidade Federal de Minas Gerais,2,"[2016, –, 2019]",2016,2019,3,Bachelor’s Degree,3.0,Universidade Federal de Minas Gerais,Computer Technology/Computer Systems Technology,TI,2,3.0,TI,3
5,2013 – 2015,Nome do diploma\nTechnician,"Área de estudo\nInformatics, COLTEC",Universidade Federal de Minas Gerais,2,"[2013, –, 2015]",2013,2015,2,Technician,2.0,Universidade Federal de Minas Gerais,"Informatics, COLTEC",TI,2,3.0,NaN,NaN
6,2014 – 2019,Nome do diploma\nEngenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3,"[2014, –, 2019]",2014,2019,5,Engenharia de Produção Civil,3.0,Centro Federal de Educação Tecnológica de Mina...,Não Informado,Engenharia,3,3.0,Engenharia,5
7,2011 – 2016,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,Universidade Federal de Minas Gerais,4,"[2011, –, 2016]",2011,2016,5,Bachelor of Science (B.S.),3.0,Universidade Federal de Minas Gerais,Chemical Engineering,Engenharia,4,3.0,Engenharia,5
8,2015 – 2015,Nome do diploma\nBachelor of Science (B.S.),Área de estudo\nChemical Engineering,California State University-Long Beach,4,"[2015, –, 2015]",2015,2015,0,Bachelor of Science (B.S.),3.0,California State University-Long Beach,Chemical Engineering,Engenharia,4,3.0,Engenharia,0
9,2011 – 2011,Nome do diploma\nPós Graduação em Gestão Estra...,Área de estudo\nEconomia e Gestão Empresarial,Universidade FUMEC,5,"[2011, –, 2011]",2011,2011,0,Pós Graduação em Gestão Estratégica de Negócios,4.0,Universidade FUMEC,Economia e Gestão Empresarial,Economia / Finanças,5,4.0,Economia / Finanças,0


In [11]:
#Área da máxima escolaridade
a=pd.DataFrame(formacao.max_area)
a['key']=formacao.perfil_id
a=a[a.max_area.notnull()]

#Mostra o reultado
a.head(10)

,max_area,key
0,Gestão / Negócios,0
2,Gestão / Negócios,1
4,TI,2
6,Engenharia,3
7,Engenharia,4
8,Engenharia,4
9,Economia / Finanças,5
11,TI,6
12,Gestão / Negócios,7
13,Gestão / Negócios,8


In [12]:
#Duração da máxima escolaridade
d=pd.DataFrame(formacao.max_duracao)
d['key']=formacao.perfil_id
d=d[d.max_duracao.notnull()]

#Mostra o resultado
d.head(10)

,max_duracao,key
0,0,0
2,0,1
4,3,2
6,5,3
7,5,4
8,0,4
9,0,5
11,6,6
12,3,7
13,0,8


In [13]:
#Gera a resposta final
resposta1=pd.merge(max_esc,a,how='outer',on=['key'])
resposta03=pd.merge(resposta1,d,how='outer',on=['key'])

#Mostra o resultado
resposta03.head(10)

,key,max_grau,max_area,max_duracao
0,0,4.0,Gestão / Negócios,0
1,1,4.0,Gestão / Negócios,0
2,2,3.0,TI,3
3,3,3.0,Engenharia,5
4,4,3.0,Engenharia,5
5,4,3.0,Engenharia,0
6,4,3.0,Engenharia,5
7,4,3.0,Engenharia,0
8,5,4.0,Economia / Finanças,0
9,6,4.0,TI,6


In [14]:
#Remove dados duplicados, ajuste de valores e cria coluna: duracao
resposta03['duracao']=resposta03.max_duracao
resposta03.loc[resposta03.max_duracao==0,'duracao']='menos de 1 ano'
resposta03=resposta03.drop_duplicates()

#Mostra o resultado
resposta03.head(10)

,key,max_grau,max_area,max_duracao,duracao
0,0,4.0,Gestão / Negócios,0,menos de 1 ano
1,1,4.0,Gestão / Negócios,0,menos de 1 ano
2,2,3.0,TI,3,3
3,3,3.0,Engenharia,5,5
4,4,3.0,Engenharia,5,5
5,4,3.0,Engenharia,0,menos de 1 ano
8,5,4.0,Economia / Finanças,0,menos de 1 ano
9,6,4.0,TI,6,6
10,7,3.0,Gestão / Negócios,3,3
11,8,4.0,Gestão / Negócios,0,menos de 1 ano


In [15]:
#Ajuste mais profundo para remover dados dublicados
resposta03=resposta03.drop([5,24,37,54,55,56,60,62,63,65,66,78,79,80,81,82,88,89,90,101,106,107,108,110,122,142,144,145,151,163,164,165,167,171,172,173,177,178,179,182,183,184])
resposta03.loc[39,'duracao']=7
resposta03.loc[100,'duracao']=5
resposta03.loc[152,'duracao']=7

#Mostra resposta
resposta03.head(10)

,key,max_grau,max_area,max_duracao,duracao
0,0,4.0,Gestão / Negócios,0,menos de 1 ano
1,1,4.0,Gestão / Negócios,0,menos de 1 ano
2,2,3.0,TI,3,3
3,3,3.0,Engenharia,5,5
4,4,3.0,Engenharia,5,5
8,5,4.0,Economia / Finanças,0,menos de 1 ano
9,6,4.0,TI,6,6
10,7,3.0,Gestão / Negócios,3,3
11,8,4.0,Gestão / Negócios,0,menos de 1 ano
12,9,4.0,Gestão / Negócios,0,menos de 1 ano


In [16]:
#Preparando a resposta
resposta03['max_grau']=resposta03['max_grau'].astype(int)
resposta03['escolaridade']= resposta03.max_grau.apply(lambda x: ['Não Informado','Ensino Médio','Técnico','Graduação','Pós-Graduação'][x]).astype("category")
resposta03=resposta03.rename(columns={'max_grau':'escolaridade_numero','max_area':'area'})
resposta03['perfil_id']=resposta03.key
resposta03=resposta03.drop(['key','max_duracao'],axis=1)
resposta03=resposta03.set_index('perfil_id')

#Mostra o resultado
resposta03.head(10)

,escolaridade_numero,area,duracao,escolaridade
perfil_id,,,,
0,4,Gestão / Negócios,menos de 1 ano,Pós-Graduação
1,4,Gestão / Negócios,menos de 1 ano,Pós-Graduação
2,3,TI,3,Graduação
3,3,Engenharia,5,Graduação
4,3,Engenharia,5,Graduação
5,4,Economia / Finanças,menos de 1 ano,Pós-Graduação
6,4,TI,6,Pós-Graduação
7,3,Gestão / Negócios,3,Graduação
8,4,Gestão / Negócios,menos de 1 ano,Pós-Graduação


## 03.6 Salvando Base Tratada

A base tratada será gravada no arquivo **formacao-tratado.csv**.

A base consolidadada com uma informação por registro será gravada no arquivo **perfil-formacao.csv** contendo as colunas: **perfil_id**, **escolaridade**, **escolaridade_numero**, **area**, **duracao**.

In [17]:
# Grava a formação tratada em arquivo
formacao_tratado=formacao[['perfil_id','inicio','fim','duracao','grau','instituicao','area']].copy().set_index('perfil_id')
formacao_tratado.to_csv('bases/formacao-tratado.csv')

# Grava o consolidade da formação por perfil em arquivo
formacao_perfil=resposta03.copy()
formacao_perfil.to_csv('bases/perfil-formacao.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)
* [Continuar para o próximo documento](04-tratamento-base-habilidades.ipynb)